In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

In [2]:
df_recipes = pd.read_csv("RAW_recipes.csv", converters={'tags': lambda x: x[1:-1].split(',')})  #https://stackoverflow.com/questions/45758646/pandas-convert-string-into-list-of-strings

In [3]:
#tags = df_recipes["tags"]#.iloc[0:1000]
df_recipes["tags"] = df_recipes["tags"].apply(lambda lst: [re.sub(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '', s).lower() for s in lst])   #chatgpt for regex...

Counter(np.concatenate(df_recipes["tags"]))

Counter({'60-minutes-or-less': 69990,
         'time-to-make': 225326,
         'course': 218148,
         'main-ingredient': 170446,
         'cuisine': 91165,
         'preparation': 230546,
         'occasion': 114145,
         'north-american': 48479,
         'side-dishes': 26902,
         'vegetables': 53814,
         'mexican': 6694,
         'easy': 126062,
         'fall': 7754,
         'holiday-event': 34920,
         'vegetarian': 35651,
         'winter': 7654,
         'dietary': 165091,
         'christmas': 8145,
         'seasonal': 21933,
         'squash': 3696,
         '30-minutes-or-less': 55077,
         'breakfast': 13655,
         'main-dish': 71786,
         'pork': 12692,
         'american': 31179,
         'oven': 31180,
         'kid-friendly': 27074,
         'pizza': 1505,
         'northeastern-united-states': 2566,
         'meat': 56042,
         'equipment': 70436,
         'chili': 1693,
         'crock-pot-slow-cooker': 6608,
         '4-hours-or-l

In [4]:
def filter_tags(df, tags):
    df_filtered = df.copy()
    tags_set = set(tags)
    df_filtered = df_filtered[~df_filtered['tags'].apply(lambda x: bool(tags_set.intersection(set(x))))]
    
    return df_filtered

df_temp = filter_tags(df_recipes,["beverages"])

## Ingredients extraction
#### LLAMA3 set up
1. Download from https://ollama.com/
2. Install
3. Click here http://localhost:11434/ to check if it’s running (it should appear 'Ollama is running')
4. Run in a terminal: `ollama run llama3`

If u dont have: `pip install langchain-community`

In [10]:
from langchain_community.llms import Ollama
from collections import Counter
from preprocessing import *
import re

llm = Ollama(model="llama3")
sentences = ["I have tomatoes, onions, and garlic. What can I cook with them?",
             "What dishes can I make with pasta, spinach, and cheese?",
             "I found some shrimp, bell peppers, and mushrooms in the fridge. Any recipe suggestions?",
             "My pantry has rice, beans, and tomatoes. What's a simple yet delicious meal I can prepare?",
             "I have eggs, bacon, and bread. How can I turn these into a tasty breakfast?",
             "What can I whip up with ground beef, bell peppers, and onions?",
             "I bought salmon, asparagus, and lemon. What's a good recipe for a healthy dinner?",
             "I've got tofu, broccoli, and soy sauce. Any suggestions for a vegetarian stir-fry?",
             "What desserts can I make with flour, sugar, and chocolate?",
             "I'm craving something sweet. What can I make with apples, cinnamon, and oats?"]


the LLM approach could look like:
prompt:
List all food ingredients from the following text {{user prompt}

In [8]:

def extractIngredients(sentence):
    output = ''
    prompt = f"Output the ingredients from the following sentence without any comments, separated only by white spaces: {sentence}"
    for chunks in llm.stream(prompt):
        output += chunks

    pattern = r'\b[a-zA-Z]+\b'  # Simple pattern to match words
    ingredients = re.findall(pattern, output.lower())
    return list(ingredients)


# Extract and count all the ingredients form a DF
def extract_info_form_df(df, info='ingredients'):
    df_food = filter_tags(df,["beverages"])
    df_food.reset_index(drop=True, inplace=True)
    ingredients = df_food[info]#.iloc[0:1000]
    all_ingredients = np.concatenate(ingredients)
    all_ingredients_counter = Counter(all_ingredients)  
    return all_ingredients_counter


In [ ]:
for sentence in sentences:
    print(f"Sentence:\n{sentence}\nIngredients:\n{extractIngredients(sentence)}\n")

Sentence:
I have tomatoes, onions, and garlic. What can I cook with them?
Ingredients:
['tomatoes', 'onions', 'garlic']

Sentence:
What dishes can I make with pasta, spinach, and cheese?
Ingredients:
['pasta', 'spinach', 'cheese']

Sentence:
I found some shrimp, bell peppers, and mushrooms in the fridge. Any recipe suggestions?
Ingredients:
['shrimp', 'bell', 'peppers', 'mushrooms']

Sentence:
My pantry has rice, beans, and tomatoes. What's a simple yet delicious meal I can prepare?
Ingredients:
['rice', 'beans', 'tomatoes']

Sentence:
I have eggs, bacon, and bread. How can I turn these into a tasty breakfast?
Ingredients:
['eggs', 'bacon', 'bread']

Sentence:
What can I whip up with ground beef, bell peppers, and onions?
Ingredients:
['ground', 'beef', 'bell', 'peppers', 'onions']

Sentence:
I bought salmon, asparagus, and lemon. What's a good recipe for a healthy dinner?
Ingredients:
['salmon', 'asparagus', 'lemon']

Sentence:
I've got tofu, broccoli, and soy sauce. Any suggestions f